In [22]:
import os
import pandas as pd
import subprocess
import re
import time


In [23]:
def identify():
    while True:
        protein_in = "_".join(input("please input your protein name\n\t").split())
        taxonomy_in = "_".join(input("please input the taxonomic group you want to analyse\n\t").split())
        if protein_in == "" or taxonomy_in == "":
            print("\n\nYou missed at least one input, I CAN'T create something out of nothing ,sorry! Please try again!!!\n\n")
            continue
        else:
            break
    return protein_in, taxonomy_in

In [24]:
# prepare a check function to see if the user want to continue or not, if user didn't input something right, notify user
def check_continue(information = "do you want to continue? \n\t"):
    while True:
        user_info = input(information + " [yes/no]\n\t")
        if user_info.upper().startswith("Y"):
            return True
        elif user_info.upper().startswith("N"):
            return False
        else: 
            print("\n\tyou didn't input correctly, please input 'yes' or 'no'! \t")

In [38]:
def download_file(protein_ask = "pyruvate_dehydrogenase", taxonomy_ask = "ascomycete_fungi"):
    while True:
        result_readable = subprocess.check_output(f"esearch -db protein -query '{protein_ask}[Protein Name] AND {taxonomy_ask}[Organism]'", shell = True).decode("utf-8")
        search_num = re.search(r"<Count>(.*?)</Count>", result_readable).group(1)
        if int(search_num) >= 1000:
            print("\nSorry but there are too many sequences found based on your search criteria, which is hard to process, could you please try again?\n")
            protein_ask, taxonomy_ask = identify()
            continue
        elif check_continue(f"you have found {search_num} sequences, do you wanna continue?"):
            print("downloading...... in a second~")
            break
        else:
            print("Since you don't want to continue, you are going to exit in 3 seconds. See you next time~")
            time.sleep(3)
            exit()
    os.system(f"esearch -db protein -query '{protein_ask}[Protein Name] AND {taxonomy_ask}[Organism]' | efetch -format fasta > {protein_ask}_{taxonomy_ask}.fa")
    return print(f"\n###{protein_ask}_{taxonomy_ask}.fa### has been successfully downloaded to your current working directory")


In [39]:
protein, taxonomy = identify()

download_file(protein, taxonomy)

please input your protein name
	dehydrogenase
please input the taxonomic group you want to analyse
	fungi

Sorry but there are too many sequences found based on your search criteria, which is hard to process, could you please try again?

please input your protein name
	pyruvate_dehydrogenase
please input the taxonomic group you want to analyse
	ascomycete_fungi
you have found 232 sequences, do you wanna continue? [yes/no]
	yes
downloading...... in a second~

###pyruvate_dehydrogenase_ascomycete_fungi.fa### has been successfully downloaded to your current working directory


In [14]:
print(protein, taxonomy)

ascomycete_fungi pyruvate_dehydrogenase


In [8]:
with open(f"{protein}_{taxonomy}.fa") as my_file:
    species = []
    acc_nums = []
    seqs = []
    current_seq = ""
    for file_line in my_file:
        if file_line.startswith(">"):
            if current_seq != "":
                seqs.append(current_seq)
            current_seq = ""
            acc_num = re.search(r">([^\s]*)", file_line)
            if acc_num == None:
                acc_num = "-"
            else:
                acc_num = acc_num.group(1)
            organism = re.search(r"\[(.*)]", file_line)
            if organism == None:
                organism = "-"
            else:
                organism = organism.group(1)
            # 如果没有搜索到相应的内容，进行置空处理
            acc_nums.append(acc_num)
            species.append(organism)
#         elif current_seq == "":
#             current_seq = file_line.strip()
        else:
            current_seq += file_line.strip()
    seqs.append(current_seq)

In [19]:
seqlen = []
i = 0
for seq in seqs:
    seqlen.append(len(seq))
species_unrep_num = len(set(species))
species_unrep = list(set(species))



In [42]:
print(f"{sorted(species_unrep)}")



['Ambrosiozyma platypodis', 'Ascobolus immersus RN42', 'Ascodesmis nigricans', 'Ascoidea rubescens DSM 1968', 'Aspergillus aculeatinus CBS 121060', 'Aspergillus arachidicola', 'Aspergillus bombycis', 'Aspergillus clavatus NRRL 1', 'Aspergillus costaricaensis CBS 115574', 'Aspergillus eucalypticola CBS 122712', 'Aspergillus fischeri NRRL 181', 'Aspergillus flavus', 'Aspergillus fumigatus Z5', 'Aspergillus heteromorphus CBS 117.55', 'Aspergillus luchuensis', 'Aspergillus luchuensis IFO 4308', 'Aspergillus neoniger CBS 115656', 'Aspergillus niger', 'Aspergillus niger ATCC 13496', 'Aspergillus niger CBS 101883', 'Aspergillus niger CBS 513.88', 'Aspergillus nomiae NRRL 13137', 'Aspergillus novofumigatus IBT 16806', 'Aspergillus phoenicis ATCC 13157', 'Aspergillus piperis CBS 112811', 'Aspergillus sclerotialis', 'Aspergillus sclerotiicarbonarius CBS 121057', 'Aspergillus sp. HF37', 'Aspergillus steynii IBT 23096', 'Aspergillus tubingensis', 'Aspergillus ustus', 'Aspergillus vadensis CBS 1133

In [31]:
all_data = {"Accession Number": acc_nums, "organisms" : species, "length of sequences" : seqlen, "Sequences" : seqs}
df_all_data = pd.DataFrame(all_data)

In [32]:
df_all_data

,Accession Number,organisms,length of sequences,Sequences
0,KAI5905415.1,Candida parapsilosis,597,MLSTRPKFLYLTNKQLQTISKRLISSTSKTSVNSSKLLNLSKTFFR...
1,GJP90915.1,Aspergillus niger,517,MLTSLGGARTRLLRVPTTSPIRHQGLVPRNQNHTRHSHSYYSTKGA...
2,GJP95986.1,Aspergillus niger,400,MLTRSLRPKGTLRWTRPGLPYRRCLAHVSNVSDLPTEENKPFSVPI...
3,GAM43544.1,Talaromyces pinophilus,423,MMLSICAARLRTRLGRVPASIFRTQTKRSLSTGKPTSNSKLYVAIA...
4,GJC93706.1,Colletotrichum higginsianum,426,MAFYEQRYLETGVGLGRLMVFGRSLTESRLLSSANYVRTELPTRIA...
...,...,...,...,...
227,KGQ11384.1,Beauveria bassiana D1-5,573,MKQSVAAWLAKTLESAGVKRIWGVTGDSLNGLSDSLNRMGTIKWMP...
228,XP_008602215.1,Beauveria bassiana ARSEF 2860,478,MFPRRVVLTRTYSSVTLQRQLRKFASKNLSPLAPGINKGLRYTAIA...
229,XP_008593908.1,Beauveria bassiana ARSEF 2860,417,MASVLQHSDDDVRHVLSQDAHSVSLHTLGVKRVDAAQLPANNPCED...
230,XP_001399196.2,Aspergillus niger CBS 513.88,544,MLIFNLTFAAPSTATNHLIYTTISLVPPMFTSFGGARTRLLRVPTT...


In [34]:
if check_continue("\n...\n...\n...\nAll the data has been organized in a form, do you wanna generate a tsv file?"):
    df_all_data.to_csv("all_data_you_searched.tsv", sep = '\t', header = True)
    print("\ntsv file named 'all_data_you_searched.tsv' has been generated in the current working dictionary")
    


...
...
...
All the data has been organized in a form, do you wanna generate a tsv file? [yes/no]
	yes

tsv file named 'all_data_you_searched.tsv' has been generated in the current working dictionary


In [33]:
df_all_data.to_csv("all_data_you_searched.tsv", sep = '\t', header = True)

In [16]:
len(seqlen)


232

In [17]:
os.system("patmatmotifs")

Scan a protein sequence with motifs from the PROSITE database
Input protein sequence: Error: Unable to get reply from user - end of standard input


256